Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/production-deploy-to-aks-gpu/production-deploy-to-aks-gpu.png)

# Deploying a web service hosted on NVIDIA Triton to Azure Kubernetes Service (AKS)
This notebook shows the steps for deploying a service with [NVIDIA Triton Inferencing Server](https://developer.nvidia.com/nvidia-triton-inference-server): registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. 
We then test and delete the service, image and model.

In [ ]:
import azureml.core
print(azureml.core.VERSION)

# Get workspace
Load existing workspace from the config file info.

In [1]:
from azureml.core.workspace import Workspace

subscription_id = os.getenv("SUBSCRIPTION_ID", default="5f08d643-1910-4a38-a7c7-84a39d4f42e0")
resource_group = os.getenv("RESOURCE_GROUP", default="yifyu")
workspace_name = os.getenv("WORKSPACE_NAME", default="triton2")
ws = Workspace.get(
    subscription_id = subscription_id,
    resource_group = resource_group,
    name = workspace_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.13.0a0 (/home/ralf/.local/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.10.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.13.0a0 (/home/ralf/.local/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.10.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.13.0a0 (/home/ralf/.local/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.10.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from

# Download the model

Prior to registering the model, you should have a model in one of the [supported formats](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#framework-model-definition) by Triton. This cell will download a [pretrained ONNX densenet](https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx) model.

** Note: ** If you have a previously-registered model, the file name needs to follow the [naming convention](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html?highlight=model%20onnx#framework-model-definition) or be specified using model configuration file below.

In [2]:
import os
import requests
import shutil
import tarfile
import tempfile

from io import BytesIO

model_url = 'https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx'

version = '1'
model_dir = os.path.join('models', 'triton', 'densenet_onnx', version)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

target_file = os.path.join(model_dir, 'model.onnx')

if not os.path.exists(target_file):
    response = requests.get(model_url)
    open(target_file, 'wb').write(response.content)

config_file = os.path.join('models', 'triton', 'densenet_onnx', 'config.pbtxt')

# Add Model Configuration file

Each model needs a [Model Configuration](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_configuration.html) that provides required and optional information about the model.


In [32]:
%%writefile $config_file
name: "densenet_onnx"
platform: "onnxruntime_onnx"
max_batch_size: 0
input [
  {
    name: "data_0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
    reshape { shape: [ 1, 3, 224, 224 ] }
  }
]
output [
  {
    name: "fc6_1"
    data_type: TYPE_FP32
    dims: [ 1000 ]
    reshape { shape: [ 1, 1000, 1, 1 ] }
    label_filename: "densenet_labels.txt"
  }
]
instance_group [
  {
    count: 16
    kind: KIND_CPU
  }
]

Overwriting models/triton/densenet_onnx/config.pbtxt


# Register the model
Register an existing trained model, add description and tags.

** Note: ** Under `model_path` there must be a sub-directory named `triton`, which has the structure of a Triton [Model Repository](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#repository-layout).

In [33]:
from azureml.core.model import Model

model = Model.register(model_path="models", # This points to the local directory to upload.
                       model_name="densenet_onnx", # This is the name the model is registered as.
                       tags={'area': "Image classification", 'type': "classification"},
                       description="Image classification trained on Imagenet Dataset",
                       workspace=ws)

print(model.name, model.description, model.version)

Registering model densenet_onnx
densenet_onnx Image classification trained on Imagenet Dataset 5


# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [11]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "aks-cluster-1"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AksCompute.provisioning_configuration(cluster_purpose=AksCompute.ClusterPurpose.DEV_TEST,
                                                           agent_count=1,
                                                           vm_size="Standard_NC6s_v3")
    
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)

Found existing gpu cluster


# Deploy the model as a web service to AKS

First create a scoring script

** Note: ** Triton server listens to a fixed local port. You may choose to use the Triton Python [client library](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/client_library.html) to talk to it, while keeping the flexibility of pre-/post- processing.

In [5]:
%%writefile score.py
import numpy as np
from PIL import Image
import sys
from functools import partial
import os
import io

import tritonhttpclient
from tritonclientutils import InferenceServerException

from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse

sys.path.append(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'models'))
from utils import preprocess, postprocess


trition_client = None
_url = "localhost:8000"
_model = "densenet_onnx"
_scaling = "INCEPTION"

def init():
    global triton_client, max_batch_size, input_name, output_name, dtype

    triton_client = tritonhttpclient.InferenceServerClient(_url)

    max_batch_size = 0
    input_name = "data_0"
    output_name = "fc6_1"
    dtype = "FP32"


@rawhttp
def run(request):
    if request.method == 'POST':
        
        reqBody = request.get_data(False)
        img = Image.open(io.BytesIO(reqBody))
        
        image_data = preprocess(img, _scaling, dtype)
        
        input = tritonhttpclient.InferInput(input_name, image_data.shape, dtype)
        input.set_data_from_numpy(image_data, binary_data=True)
        output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=True, class_count=1)
    
        res = triton_client.infer(_model,
                                [input],
                                request_id="0",
                                outputs=[output])

        result = postprocess(res, output_name, 1, max_batch_size > 0)

        return AMLResponse(result, 200)
    else:
        return AMLResponse("bad request", 500)

Overwriting score.py


** Note: ** If you use the Triton Python client, download the Wheels and include them in the `Environment`.

In [6]:
triton_client_url = 'https://github.com/NVIDIA/triton-inference-server/releases/download/v2.1.0/v2.1.0_ubuntu1804.clients.tar.gz'

client_filename = 'tritonclientutils-2.1.0-py3-none-any.whl'
clientutils_filename = 'tritonhttpclient-2.1.0-py3-none-any.whl'
target_folder = 'python_client'

if not os.path.exists(target_folder):
    response = requests.get(triton_client_url)
    archive = tarfile.open(fileobj=BytesIO(response.content))
    with tempfile.TemporaryDirectory() as temp_folder:
        archive.extractall(temp_folder)
        os.mkdir(target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', client_filename), target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', clientutils_filename), target_folder)

Now create the deployment configuration objects and deploy the model as a webservice.

In [37]:
# Set the web service configuration (using default here)
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

env = Environment("deploytocloudenv")
conda_dep = CondaDependencies("env.yml")
client_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, client_filename), exist_ok=True)
clientutils_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, clientutils_filename), exist_ok=True)
conda_dep.add_pip_package(client_whl_url)
conda_dep.add_pip_package(clientutils_whl_url)
env.python.conda_dependencies = conda_dep

# Specify the Azure ML Triton base image
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-nvidia-tritonserver20.07-py3'

# Optionally specify a worker count to leverage the capability of concurrency and server-side batching from Triton
env.environment_variables = {"WORKER_COUNT":"128"}

inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration(cpu_cores = 30, memory_gb = 40, auth_enabled=False, num_replicas=1, replica_max_concurrent_requests=16)

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4, gpu_cores = 1, token_auth_enabled=True, auth_enabled=False)

In [40]:
%%time
aks_service_name ='densenet-onnx-triton-1'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=gpu_cluster)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Running.........
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 154 ms, sys: 19.2 ms, total: 173 ms
Wall time: 57.6 s


# Test the web service
We test the web sevice by passing the test images content.

In [31]:
%%time
import requests

# if (key) auth is enabled, fetch keys and include in the request
#key1, key2 = aks_service.get_keys()

#headers = {'Content-Type':'application/octet-stream', 'Authorization': 'Bearer ' + key1}
headers = {'Content-Type':'application/octet-stream'}

# # if token auth is enabled, fetch token and include in the request
# access_token, fetch_after = aks_service.get_token()
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

test_sample = open('car.jpg', 'rb').read()
resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)
print(resp.text)

12.989693 (817) = SPORTS CAR
CPU times: user 6.77 ms, sys: 714 µs, total: 7.49 ms
Wall time: 365 ms


In [15]:
print(aks_service.scoring_uri)

http://40.78.16.120:80/api/v1/service/densenet-onnx-triton/score


# Clean up
Delete the service, image, model and compute target

In [ ]:
%%time
aks_service.delete()
model.delete()
gpu_cluster.delete()

In [41]:
aks_service.delete()
